In [24]:
import os
import sys
import datetime
import gspread
import pandas as pd
from urllib.parse import quote
from oauth2client.service_account import ServiceAccountCredentials
from sqlalchemy import create_engine
from sqlalchemy import select

## Function ## match result 유효한 데이터 값만 가져오기
def gt_matchRecord(st_data):
    rows = list()
    for x in range(3,30):
        if st_data[x][2] == '-':
            break
        extend_row = list(st_data[x][0:15])
        rows.append(extend_row)
    return rows

def gt_matchSummary(st_data):
    rows = list()
    for x in range(3,30):
        if st_data[x][20] == '':
            break
        else:
            if st_data[x][21] == '':
                pass
            else:
                extend_row = list(st_data[x][20:29])
                rows.append(extend_row)
    return rows    

def insert_matchSummary(pd_rows, match_Num, match_Date):
    matchdata = pd.DataFrame(pd_rows, columns= ["team","name","win","draw","lose","득점","실점","goal","assist"])
    matchdata.insert(0,'mnum',match_Num)
    matchdata.insert(1,'mdate',match_Date)
    print(matchdata)

    # DB ORM 연결, 패스워드 특수문자 치환 #
    engine = create_engine('postgresql://app_fs:%s@localhost:5432/db_fs' % quote('pok1234@'),echo = True) 
    # engine.execute("DROP TABLE IF EXISTS public.tb_test;") # drop table if exists
    # data insert to DBMS Table #
    matchdata.to_sql(
            name = 'tb_matchsummary',
            con = engine,
            schema = 'public',
            if_exists = 'append',
            index = False
            )
    engine.dispose()
    return

# Google API 연결 설정 #
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive',]
work_path = os.getcwd()
resource_path = os.path.join(work_path,'resources') # resources 디렉토리 path 설정
json_file_name = os.path.join(resource_path,'quiet-amp-275114-083a1f3b2f00.json')              
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

# DB ORM 연결, 패스워드 특수문자 치환 #
engine = create_engine('postgresql://app_fs:%s@localhost:5432/db_fs' % quote('pok1234@'),echo = True) 
result = engine.execute("select * from tb_sheeturls")
url_data = result.fetchall()
engine.dispose()

# 구글시트 url 리스트 #
spreadsheet_url = url_data[0][0] # 1경기 URL
match_NumAndDate = url_data[0][1]
match_Num = match_NumAndDate[:3]
format = '%Y-%m-%d'
match_Date = datetime.datetime.strptime(match_NumAndDate[-8:], "%Y%m%d").date()

# 스프레스시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
worksheet = doc.sheet1  ## 시트 선택하기
sheet_data = worksheet.get_all_values()  ## 시트 전체 값을 List of List 로 저장

# matchSummary Pandas에 저장 #
pd_rows = gt_matchSummary(sheet_data)

# DB Insert #
insert_matchSummary(pd_rows, match_Num, match_Date)



2022-03-17 23:48:33,845 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-03-17 23:48:33,846 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-17 23:48:33,847 INFO sqlalchemy.engine.Engine select current_schema()
2022-03-17 23:48:33,848 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-17 23:48:33,848 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-03-17 23:48:33,848 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-17 23:48:33,849 INFO sqlalchemy.engine.Engine select * from tb_sheeturls
2022-03-17 23:48:33,849 INFO sqlalchemy.engine.Engine [raw sql] {}
   mnum       mdate team name win draw lose  득점  실점 goal assist
0   M01  2021-07-04    A  장근호   5    0    7  14  16    0      1
1   M01  2021-07-04    A  이민규   5    0    7  14  16    3      0
2   M01  2021-07-04    A  박성국   5    0    7  14  16    1      3
3   M01  2021-07-04    A  이인호   5    0    7  14  16    5      1
4   M01  2021-07-04    A  장지훈   5    0    7  14  16    2      0
5   M01  2021-07-04 

In [21]:
import os
import sys
import datetime
import pandas as pd
from urllib.parse import quote
from sqlalchemy import create_engine
from sqlalchemy import select

csv = pd.read_csv("./resources/mom.csv")

# 투표자 list 만들고 투표자 이름 저장
voter = list()
voter = csv.iloc[0]
type(voter)

print(voter[0])
print(voter)

match_date = csv.loc[0]
#print(match_date)

이름
0             이름
1     2021-07-04
2     2021-07-11
3     2021-07-18
4     2021-07-25
5     2021-11-07
6     2021-11-14
7     2021-11-21
8     2021-11-28
9     2021-12-12
10    2021-12-19
11    2021-12-26
12    2022-01-09
13    2022-01-16
14    2022-01-23
15    2022-02-06
16    2022-02-13
17    2022-02-20
18    2022-02-27
19    2022-03-06
20    2022-03-13
21    2022-03-27
22    2022-04-10
23    2022-04-17
24    2022-04-24
25    2022-05-01
26    2022-05-15
27    2022-05-22
28    2022-05-29
29           NaN
30           NaN
31         Total
32           NaN
Name: 0, dtype: object
